In [ ]:
import ipinfo
import csv
from flask import Flask, render_template, request, session, redirect, url_for, send_file
import os
from tqdm import tqdm
from flask import Flask, request, render_template, redirect, url_for, send_from_directory
from werkzeug.utils import secure_filename
from datetime import datetime
import folium
import pandas as pd
import glob

app = Flask(__name__, template_folder="C:\\Users\\SAL\\Documents\\ip_mapping_final\\ip_mapping_final\\ip_mapping\\Flask-Web-App-IP-Address\\templates\\")

UPLOAD_FOLDER = "C:\\Users\\SAL\\Documents\\ip_mapping_final\\ip_mapping_final\\ip_mapping\\Flask-Web-App-IP-Address\\uploads\\"
ALLOWED_EXTENSIONS = 'csv'
# upload directory file config
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

def getloc(ip):
    access_token = 'e7b96a2a6d1cd4'
    handler = ipinfo.getHandler(access_token)
    ip_address = ip
    try:
        details = handler.getDetails(ip_address)
        return details.all
    except:
        return {}


app.secret_key = "fiu"


@app.route("/")
def home(): 
    return render_template('home.html')

@app.route("/SingleSearch", methods=['GET', 'POST'])
def SingleSearch():
    city = ''
    state = ''
    d = {}
    lat = 0.0
    lon = 0.0
    if request.method == 'POST':
        session['ip'] = request.form.get('ip')
        d = getloc(session['ip'])
        if d:
            lat = d['latitude']
            lon = d['longitude']
            
    return render_template('index.html', ip=session['ip'] if 'ip' in session else None,
                           ifInvalid='Invalid IP' if not d else '', list=d, lat=lat, lon=lon)

@app.route("/MultipleSearch", methods=['GET', 'POST'])
def multisearch():
    return render_template('index_2.html')

@app.route('/download',methods=["GET","POST"])
def download():
    list_of_files = glob.glob('C:\\Users\\SAL\\Documents\\ip_mapping_final\\ip_mapping_final\\ip_mapping\\Flask-Web-App-IP-Address\\data\\*')
    latest_file = max(list_of_files, key=os.path.getctime)
    return send_from_directory(latest_file,cache_timeout=0)

@app.route("/upload", methods = ['GET','POST'])
def upload():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            new_filename = f'{filename.split(".")[0]}.csv'
            save_location = os.path.join(os.path.join(app.config['UPLOAD_FOLDER'], new_filename))
            file.save(save_location)
        ip_df = pd.read_csv(save_location)
        ip_dic = {'latitude' : [], 'longitude': [], 'city' : [], 'region' : [], 'country' : [], 'org' : [], 'ip' : [],'country_name' : []}
        for i in tqdm(ip_df['IP']):
            d = getloc(i)
            if d.get("longitude") is not None:
                ip_dic['longitude'].append(d['longitude'])
            else:
                 ip_dic['longitude'].append('Not Found')
            if d.get("latitude") is not None:
                ip_dic['latitude'].append(d['latitude'])
            else:
                ip_dic['latitude'].append('Not Found')
            if d.get("city") is not None:
                ip_dic['city'].append(d['city'])
            else:
                ip_dic['city'].append('Not Found')
            if d.get("region") is not None:
                ip_dic['region'].append(d['region'])
            else:
                ip_dic['region'].append('Not Found')
            if d.get("country") is not None:
                ip_dic['country'].append(d['country'])
            else:
                ip_dic['country'].append('Not Found')
            if d.get("org") is not None:
                ip_dic['org'].append(d['org'])
            else:
                ip_dic['org'].append('Not Found')
            if d.get("ip") is not None:
                ip_dic['ip'].append(d['ip'])
            else:
                ip_dic['ip'].append('Not Found')
            if d.get("country_name") is not None:
                ip_dic['country_name'].append(d['country_name'])
            else:
                ip_dic['country_name'].append('Not Found')
        final_ip_df = pd.DataFrame().from_dict(ip_dic)
        map_df = final_ip_df[final_ip_df['latitude'] !='Not Found']
        maps = folium.Map(location=[map_df['latitude'].astype('float').mean(), map_df['longitude'].astype('float').mean()], zoom_start=3,
                         tiles= 'OpenStreetMap')
        for pt in zip(map_df['latitude'], map_df['longitude']):
            marker = folium.Marker([pt[0], pt[1]]) #latitude,longitude
            maps.add_child(marker)
        maps.save("C:\\Users\\SAL\\Documents\\ip_mapping_final\\ip_mapping_final\\ip_mapping\\Flask-Web-App-IP-Address\\maps\\{}.html".format(filename.split(".")[0]))
        download_path = "C:\\Users\\SAL\\Documents\\ip_mapping_final\\ip_mapping_final\\ip_mapping\\Flask-Web-App-IP-Address\\data\\{}.csv".format(filename.split(".")[0])
        map_df.to_csv(download_path, index = False)
        with open("C:\\Users\\SAL\\Documents\\ip_mapping_final\\ip_mapping_final\\ip_mapping\\Flask-Web-App-IP-Address\\maps\\{}.html".format(filename.split(".")[0]), "r") as f:
            text= f.read()
            map_with_download = '''<a href="/download">Download</a>'''+text
                  
        with open("C:\\Users\\SAL\\Documents\\ip_mapping_final\\ip_mapping_final\\ip_mapping\\Flask-Web-App-IP-Address\\templates\\{}.html".format(filename.split(".")[0]), "w") as file:
            file.write(str(map_with_download))
    return render_template("{}.html".format(filename.split(".")[0]))

if __name__ == "__main__":
    
    app.run(debug = True, host = 'localhost',use_reloader = False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Sep/2023 13:58:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2023 13:58:23] "GET /static/home.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Sep/2023 13:58:26] "GET /SingleSearch HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2023 13:58:26] "GET /static/css/sb-admin-2.min.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2023 13:58:27] "GET /static/vendor/jquery/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2023 13:58:27] "GET /static/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2023 13:58:34] "GET /MultipleSearch HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2023 13:58:34] "GET /static/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Sep/2023 13:58:54] "POST /upload HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\SAL\anaconda3\Lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^